In [ ]:
import requests
import os
from dotenv import load_dotenv
import time

# 1. API 키 로딩
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
search_url = "https://tools.kinds.or.kr/search/news"

# 2. 설정
query = "인공지능 OR AI"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 50

# 3. 뉴스 검색 API 호출
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "provider": ["서울경제"],
        "category": [],
        "category_incident": [],
        "sort": { "date": "desc" },
        "return_from": 0,
        "return_size": fetch_count,
        "fields": ["title", "news_id", "published_at"]
    }
}

search_response = requests.post(search_url, json=search_payload)

# 4. 기사 리스트 추출
if search_response.status_code == 200 and search_response.json().get("result") == 0:
    search_result = search_response.json()
    search_docs = search_result["return_object"]["documents"]
    news_ids = [doc["news_id"] for doc in search_docs]
    print(f"✅ 총 {len(news_ids)}개의 뉴스 수집 완료!\n")
else:
    print("❌ 뉴스 검색 실패")
    print(search_response.text)
    exit()

# 5. 뉴스 조회 API 호출 (본문 확보)
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}

time.sleep(1)
view_response = requests.post(search_url, json=view_payload)

# 6. 결과 처리 및 출력
if view_response.status_code == 200 and view_response.json().get("result") == 0:
    view_docs = view_response.json()["return_object"]["documents"]
    
    for i, doc in enumerate(view_docs, 1):
        title = doc.get("title", "").strip()
        content = doc.get("content", "").strip()
        print(f"\n{i}. 📰 제목: {title}")
        print(f"✍ 본문(앞부분): {content[:300]}...\n")
else:
    print("❌ 뉴스 본문 조회 실패")
    print(view_response.text)


❌ 뉴스 검색 실패
{"request_id": "","result": -1,"reason": "Invalid Access Key!:73c8b2bb-0c12-42a3-a010-da81937f6809"}


NameError: name 'news_ids' is not defined

: 

In [4]:
import requests, os, time
from dotenv import load_dotenv

# STEP 0: 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"
keyword_url = "https://tools.kinds.or.kr/keyword"
trend_url = "https://tools.kinds.or.kr/time_line"
word_url = "https://tools.kinds.or.kr/word_cloud"

query = "인공지능 OR AI"
from_date = "2025-04-01"
until_date = "2025-04-14"
count = 50

# STEP 1: 뉴스 수집
print("📥 STEP 1: 뉴스 수집 중...\n")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": count,
        "return_from": 0,
        "fields": ["title", "news_id"]
    }
}
resp = requests.post(search_url, json=search_payload)
docs = resp.json()["return_object"]["documents"]
news_ids = [d["news_id"] for d in docs]

# STEP 2: 본문 확보
print("📄 STEP 2: 본문 확보 중...\n")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
time.sleep(1)
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

# STEP 3: 키워드 추출
print("🔍 STEP 3: 키워드 추출 중...\n")
for i, article in enumerate(articles, 1):
    title = article["title"]
    content = article["content"]
    print(f"{i}. 📰 {title[:60]}...")

    # 특성 추출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    f = requests.post(feature_url, json=feature_payload).json()
    f_keywords = []
    if f.get("return_object"):
        raw = f["return_object"]["result"]["title"]
        f_keywords = [k.split("|")[0] for k in raw.split(" ") if "|" in k]
    print(f"🔑 특성 키워드: {f_keywords}")

    # 키워드 추출
    keyword_payload = {
        "access_key": API_KEY,
        "argument": {
            "text": content
        }
    }
    k = requests.post(keyword_url, json=keyword_payload).json()
    k_keywords = [w["keyword"] for w in k.get("return_object", {}).get("result", [])]
    print(f"🧠 본문 키워드: {k_keywords[:5]}...\n")
    time.sleep(0.3)

# STEP 4: 트렌드 분석
print("📈 STEP 4: 트렌드 분석 (날짜별 언급량)\n")
trend_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "interval": "day",
        "normalize": "false"
    }
}
t = requests.post(trend_url, json=trend_payload).json()
timeline = t["return_object"]["time_line"]
for item in timeline:
    print(f"{item['label']} → {item['hits']}건")

# STEP 5: 연관어 분석
print("\n🕸 STEP 5: 연관어 분석 결과 (AI 키워드 연관 단어 Top)\n")
word_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "AI",
        "published_at": {
            "from": from_date,
            "until": until_date
        }
    }
}
w = requests.post(word_url, json=word_payload).json()
words = w["return_object"]["nodes"]
words = sorted(words, key=lambda x: x["weight"], reverse=True)
for i, node in enumerate(words[:10], 1):
    print(f"{i}. 🔗 {node['name']} (연관 점수: {node['weight']:.3f})")


📥 STEP 1: 뉴스 수집 중...

📄 STEP 2: 본문 확보 중...

🔍 STEP 3: 키워드 추출 중...

1. 📰 JB금융·BNK금융·iM금융, AI 거버넌스 수립 추진...
🔑 특성 키워드: ['거버넌스', 'AI_거버넌스_수립_추진', 'AI', 'AI_거버넌스', '수립_추진', 'JB', 'iM', 'BNK', '금융', '수립', '추진']
🧠 본문 키워드: []...

2. 📰 경동시장에서 LG AI 세탁·건조 기술 체험하세요...
🔑 특성 키워드: ['LG', 'LG_AI_세탁', 'LG_AI', 'AI', '경동시장', '건조', '세탁', '체험']
🧠 본문 키워드: []...

3. 📰 콕스웨이브, 인도 '레난'과 AI 솔루션 개발 맞손 [이번주 스타트업]...
🔑 특성 키워드: ['AI_솔루션', 'AI', 'AI_솔루션_개발', '스타트업', '레난', '인도_레난', '이번주_스타트업', '콕스웨이브', '솔루션', '인도']
🧠 본문 키워드: []...

4. 📰 백악관도 'AI 정부' 전환 착수...
🔑 특성 키워드: ['AI_정부', 'AI', '전환_착수', '백악관', '정부', '전환', '착수']
🧠 본문 키워드: []...

5. 📰 요추협착증, 꼭 비싼 MRI 찍어야 할까? AI 썼더니[헬시타임]...
🔑 특성 키워드: ['헬시타임', 'AI', 'AI_헬시타임', 'MRI', '요추협착증']
🧠 본문 키워드: []...

6. 📰 가천대 인공지능학과 졸업생들, 후배위해 대학 발전기금 1900만원 쾌척...
🔑 특성 키워드: ['발전기금', '가천대', '대학_발전기금', '대학_발전기금_1900만', '가천대_인공지능학과', '가천대_인공지능학과_졸업생들', '지능학', '졸업생들', '1900만', '쾌척', '대학', '인공', '후배']
🧠 본문 키워드: []...

7. 📰 KT, AI로 1300만 고객 통신품질 선제적 개선한다...
🔑 특성 키워드: ['선제적', '1300만_고객', '통신품질_